# Experiment #10:  Information Retrieval
<b>Mohammed Abed Alkareem</b>
<b>1210708</b>

## 1.2.1  Installation

In [1]:
# !pip install whoosh

Defaulting to user installation because normal site-packages is not writeable


## 1.2.2  Preparing the data

In [6]:
# !pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105796 sha256=fca6b4b6919849e8f5b79649b03d2b3e1cc43cbc21276f4a55871f57d187947d
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\46\d2\26\84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle


In [7]:
# !kaggle datasets download -d stackoverflow/stacksample

Dataset URL: https://www.kaggle.com/datasets/stackoverflow/stacksample
License(s): other




  0%|          | 0.00/1.11G [00:00<?, ?B/s]
  0%|          | 1.00M/1.11G [00:00<14:39, 1.35MB/s]
  0%|          | 2.00M/1.11G [00:00<07:18, 2.71MB/s]
  0%|          | 4.00M/1.11G [00:01<04:30, 4.39MB/s]
  1%|          | 6.00M/1.11G [00:01<02:58, 6.62MB/s]
  1%|          | 8.00M/1.11G [00:01<02:41, 7.31MB/s]
  1%|          | 10.0M/1.11G [00:01<02:32, 7.74MB/s]
  1%|          | 11.0M/1.11G [00:02<02:45, 7.13MB/s]
  1%|          | 13.0M/1.11G [00:02<02:24, 8.14MB/s]
  1%|▏         | 15.0M/1.11G [00:02<02:12, 8.89MB/s]
  1%|▏         | 16.0M/1.11G [00:02<02:44, 7.13MB/s]
  2%|▏         | 18.0M/1.11G [00:02<02:23, 8.14MB/s]
  2%|▏         | 19.0M/1.11G [00:02<02:26, 8.00MB/s]
  2%|▏         | 20.0M/1.11G [00:03<02:27, 7.91MB/s]
  2%|▏         | 21.0M/1.11G [00:03<03:10, 6.13MB/s]
  2%|▏         | 22.0M/1.11G [00:03<03:07, 6.23MB/s]
  2%|▏         | 23.0M/1.11G [00:03<02:58, 6.54MB/s]
  2%|▏         | 24.0M/1.11G [00:03<02:51, 6.79MB/s]
  2%|▏         | 25.0M/1.11G [00:03<02:42, 7.18MB/s]
 

In [8]:
# !unzip stacksample.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
import pandas as pd
questions=pd.read_csv("stacksample/Questions.csv", nrows=20000)
questions

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
...,...,...,...,...,...,...,...
19995,1114470,82266.0,2009-07-11T19:37:06Z,NaN,0,"Trim all chars off file name after first ""_""",<p>I'd like to trim these purchase order file ...
19996,1114540,2288585.0,2009-07-11T20:16:06Z,NaN,7,Xcode question: Quickly jump to a particular s...,<p>What is the quickest way to jump to a parti...
19997,1114550,131128.0,2009-07-11T20:20:11Z,NaN,3,Serializing a generic collection with XMLSeria...,<p>Why won't XMLSerializer process my generic ...
19998,1114580,87271.0,2009-07-11T20:35:46Z,NaN,1,Using Yahoo Fire Eagle on Grails / Java,<p>Has anyone implemented the Yahoo Fire Eagle...


## 1.2.3  The Index and Schema objects

In [11]:
from whoosh.fields import Schema, TEXT, ID
# Defining index schema
schema = Schema(Id=ID(stored=True), Title=TEXT(stored=True),Body=TEXT(stored=True))

In [12]:
import os.path
index_dir = "indexdir"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)

In [13]:
from whoosh.index import create_in
from whoosh.index import open_dir

# Creating the index
ix = create_in(index_dir, schema)

# Open the index writer
writer = ix.writer()

# Iterate over the DataFrame and add documents to the index
# we have indexed title, title_body and doc_id
for index, row in questions.iterrows():
    writer.add_document(Id=str(row['Id']), Title = row['Title'],Body=row['Body'])
    
# Commit and close the writer
writer.commit()

## 1.2.4  How to search

In [14]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring

# create the query parser
qp = QueryParser("Title", schema=schema)

# parse the query
query_sentence = "How to install"
query = qp.parse(query_sentence)

# create a searcher object
searcher_tfidf = ix.searcher(weighting=scoring.TF_IDF())

# search documents and store them
# # we are returing top 3 documents
results_tfidf = searcher_tfidf.search(query, limit=3, scored=True)

# print the documents
for hit in results_tfidf:
    print(hit["Id"])
    print('\n')
    print(hit["Title"])
    print('\n')
    print('------------------\n')

102850


How can I install CPAN modules locally without root access (DynaLoader.pm line 229 error)?


------------------

145900


How can I determine that Windows Installer is performing an upgrade rather than a first time install?


------------------

351640


How to install Hibernate Tools in Eclipse?


------------------

